# SCRIPT to scrape DATA from AMAZON
### Author of script : SHIVAM SHARMA
### Note: Don't rely on this script since Amazon keeps changes there web layout
##### LINKEDIN : https://linkedin.com/in/shivam-sharma-6499061a9

##### GITHUB: https://github.com/Shivam38391



In [3]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import requests
import time
import csv
import re

# generating url

In [4]:
def get_url(search_product):
    """Generate a url from search"""
    template = "https://www.amazon.in/s?k={}"
    search_product =search_product.replace(' ', '+')
    
    # add term query to url
    url = template.format(search_product)
    
    url = url+ '&page={}'
    
    return url

# extracting data from each item

In [5]:
def extract_data(item):
    """extract and return data from a single record"""
    
    
    atag = item.h2.a    
    product_name = atag.text.strip()  
    product_url ='https://www.amazon.in' + atag.get("href") 
    try:
        parent_price = item.find('span', 'a-price')    
        price = parent_price.find('span' , "a-offscreen").text 
    except AttributeError:
        return
    
    try:
        rating = item.i.text
    except AttributeError:
        rating = None
        
    try:
        reviews = item.find('span', {'class': 'a-size-base s-underline-text'}).text
    except AttributeError:
        reviews = None
    
    result = (product_url , product_name, price, rating, reviews)
    
    return result

In [6]:
def main(search):
    n = int(input("Number of pages you want to scrape"))
    
    records = []
    h ={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}    

    url = get_url(search)

    for i in range(1,n):
        url = url.format(i)
#         r = requests.get(url, headers= h)  #without session
        
        r = s.get(url, headers= h)  # without header session also doesn't work
        
    
        soup = BeautifulSoup(r.content, 'html.parser')
    
        results = soup.find_all('div', attrs = {'class':'sg-col sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20 s-list-col-right'})    
        for i in results:
            data=extract_data(i)
            if data:
                records.append(data)
            
    return records

In [7]:
s = requests.session()
start_time = time.time()
main_result = main("bags")
end_time = time.time() - start_time
print(f'Time taken by the script to extract data from pages is {end_time} ')

Number of pages you want to scrape10
Time taken by the script to extract data from pages is 16.419219970703125 


In [8]:
number_of_links= len(main_result)
number_of_links

198

In [11]:
main_result

[('https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1MjYxNzU2NjQ4MDQzNzE0OjE2NzU0NDUwMDg6c3BfYXRmOjIwMTEwMTgxNTExNTk4OjowOjo&url=%2FThe-Backpack-Pro-Deep-Dive%2Fdp%2FB08SWDK9S7%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dbags%26qid%3D1675445008%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1',
  'MOKOBARA Backpack Pro 15.6" Inch Laptop Backpack For Men And Women (21 litre).',
  '₹5,990',
  '4.4 out of 5 stars',
  '223'),
 ('https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1MjYxNzU2NjQ4MDQzNzE0OjE2NzU0NDUwMDg6c3BfYXRmOjIwMTA0MTQwMjYzNjk4OjowOjo&url=%2FUrban-Tribe-Laptop-Backpack-Havana%2Fdp%2FB01LXNNFDF%2Fref%3Dsr_1_2_sspa%3Fkeywords%3Dbags%26qid%3D1675445008%26sr%3D8-2-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1%26smid%3DA385M0TPSNV7VS',
  'URBAN TRIBE Havana 15.6" inch Laptop Backpack for Men and Women | 27L Office/Work/Casual Bag | Water Repellent | Black',
  '₹1,169',
  '3.9 out of 5 stars',
  '684'),
 ('https://www.amazon.in/American-Tourister-AMT-SCH-02/dp/B07CJCGM1M/ref=s

# this will generate csv file of main result

In [ ]:
with open('main_results.csv' , 'w', newline='', encoding='utf-8') as f:
    write = csv.writer(f)
    write.writerow(['P-links', 'P-name', 'Price', 'Rating', 'Reviews'])
    write.writerows(main_result)

In [11]:
main_result[7][0]

'https://www.amazon.in/TRUE-HUMAN-Anti-Theft-backpack-charging/dp/B09QJ7ZF2Y/ref=sr_1_8?keywords=bags&qid=1675449605&sr=8-8'

# script for scraped data for each link

In [17]:
def item_extract(link):
    h ={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}    
    
    pr = s.get(link, headers= h)    
    psoup = BeautifulSoup(pr.content, 'html.parser')
    
    try:
        p_name = psoup.find('span',attrs={'id':'productTitle' , 'class': 'a-size-large product-title-word-break'}).text.strip()
    except AttributeError:
        p_name = None
    
    try:
        p_desc = psoup.find('div',attrs={'id':'feature-bullets' , 'class': 'a-section a-spacing-medium a-spacing-top-small'}).text.strip()
        
    except AttributeError:
        p_desc = None
    
    try:
        lilist = psoup.find('div', attrs={'id':'detailBullets_feature_div'}).ul    
        """ to get the Asin no of product"""
        i = 1
        res = " "
        p_asin = " "
        while res != 'ASIN  ':
            str = lilist.find_all('span', attrs={'class':'a-list-item'})[i].text
            str_en = str.encode("ascii", "ignore")
            str_de = str_en.decode()
            res = re.sub(' +', ' ', str_de)
            p_asin = res.replace('\n','').strip().split(':')[1]
            res = res.replace('\n','').strip().split(':')[0]
            i = i+1

        ##########to get manufacture########
        j = 0
        resm = " "
        p_manuf = " "
        
        while j < len(lilist):
            try:
                str = lilist.find_all('span', attrs={'class':'a-list-item'})[j].text
            except IndexError:
                str = 'Manufacturer  : Notavailable'
            str_en = str.encode("ascii", "ignore")
            str_de = str_en.decode()
            resm = re.sub(' +', ' ', str_de)
            p_manuf = resm.replace('\n','').strip().split(':')[1]
            resm = resm.replace('\n','').strip().split(':')[0]
            j = j+1
            if resm == "Manufacturer  ":
                break
                
    except AttributeError:
        p_asin = None
        p_manuf = None

            
    result = (p_name , p_desc, p_manuf ,p_asin)
    
    return result

### have some patiences it take time a while to execute  (depends on number of url you want to hit)

In [19]:
start_time = time.time()
s = requests.session()
final_list = []
for row in main_result: #usiong session 
    final_list.append(item_extract(row[0]))
    
end_time = time.time() - start_time

print(f'Time taken by the script to extract data from {number_of_links} links is {end_time} ')

Time taken by the script to extract data from 198 links is 595.4907658100128 


In [18]:
x = item_extract(main_result[7][0])
x

('TRUE HUMAN® Emperor Anti-Theft backpack With USB charging Port and anti -theft back pocket Laptop bag ,office bag ,college bags,travel bag ( legend )',
 'Superior & Durability: Made of durable and Eco-friendly water resistant oxford textile fabric,which keeps your files and stuffs dry inside. This backpack can hold up to 15.6" laptop from most brands, but it may not be suitable for all brand 15.6". Since there maybe exist slight size difference among different laptop brands.',
 ' Notavailable',
 '  B09QJ7ZF2Y')

In [21]:
len(final_list)

198

In [22]:
final_list

[('FIDO-45 Liter (Black) Fully Padded Laptop Office/School/Travel/Business Backpack Water Resistant - Fits Up to 15.6 Inch Laptop Notebook with 1 Year Warranty',
  'WATER RESISTANT WITH 1 YEAR WARRANTY - Stylish and durable unisex travel laptop backpack that is your perfect travel partner. Size: 20 inch X 13.5 inch X 9.5 inch. Capacity : 48 liters. Backpack is made of high grade water resistant polyester material that gives it strength and durable for long-term use. 12 months warranty for any manufacturing defects    ?MADE IN INDIA - This “MAKE IN INDIA” Product features 3 large zippered compartments, 1 zip pocket at front, 2 side bottle pockets. This is a perfect laptop, office, school, college bag for men, women, boys and girls    ?PADDED LAPTOP SLEEVE AND INTERNAL ORGANISER : Padded Laptop sleeve with holder to secure upto 15.6" Laptops. Compatible with laptops like HP / Lenovo / Acer / ASUS / Dell. Internal organiser to neatly organise and store your keys, cables, stationary and ot

# Script to generate final Csv file contains, ASIN , Manufacture , name and desc of product

In [23]:
with open('final_results.csv' , 'w', newline='', encoding='utf-8') as f:
    write = csv.writer(f)
    write.writerow(['Pd_name', 'Pd-desc', 'Pd-manufacture', 'ASIN'])
    write.writerows(final_list)